In [2]:
# Specify model here
from src.models import Simple1d as model


def get_parameter_count(model):
    # Dictionary to store parameters for each layer
    layer_params = {}

    # Collect parameters for each layer
    for name, param in model.named_parameters():
        if param.requires_grad:
            # Extract the layer name (everything before the last dot)
            layer_name = name.rsplit(".", 1)[0]
            num_params = param.numel()
            layer_params[layer_name] = layer_params.get(layer_name, 0) + num_params

    # Print total parameters for each layer
    print("Parameters per layer:")
    print("-" * 40)
    for layer_name, total_params in layer_params.items():
        print(f"{layer_name:25} {total_params:>10,d}")
    print("-" * 40)
    print(f"{'Total':25} {sum(layer_params.values()):>10,d}")


get_parameter_count(model())

Parameters per layer:
----------------------------------------
block0.0                         256
block1.0                      12,320
block1.1                       5,152
block2.0                       6,208
block2.1                      20,544
block3.0                      24,704
block3.1                      82,048
dense1                         1,032
dense2                            18
----------------------------------------
Total                        152,282
